### Natural Text to SQL Query

In [1]:
db_user = "root"
db_password = "9359635906"
db_host = "localhost"
db_name = "classicmodels"


In [2]:
# connection to db
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}") # by using sqlalchemy


In [3]:
print(db.dialect)
print(db.get_usable_table_names)
print(db.table_info)


mysql
<bound method SQLDatabase.get_usable_table_names of <langchain_community.utilities.sql_database.SQLDatabase object at 0x00000190F94EFF50>>

CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAINT customers_ibfk_1 FOREIGN KEY(`salesRepEmployeeNumber`) REFERENCES employees (`employeeNumber`)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from customers table:
customerNumber	customerName	contactLastName	contactFirstName	phone	addressLine1	addressLine2	city	state	postalCode	country	salesRepEmployeeNumber	credi

In [4]:
# langsmith + langchain integration credentials
import os

os.environ['COHERE_API_KEY'] = "MiOl2nhufDN8fYW7yFsKHnwxP8K5TsMhza3IJaTF"
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ['LANGCHAIN_ENDPOINT']='https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT']='Text to SQL Query'


In [5]:
from langchain.chains import create_sql_query_chain
from langchain_cohere import ChatCohere

llm = ChatCohere(model='command-r', temperature=0)
generate_query = create_sql_query_chain(llm, db)

query = "What is the price of '1968 ford mustang'. Provide SQL query as a output nothing else"
config = {
    'run_name': 'query_generation'
}
res = generate_query.invoke({'question': query}, config=config)
print(res)


Question: What is the price of '1968 ford mustang'?

SQLQuery: 
```sql
SELECT `MSRP` FROM products WHERE `productName` LIKE '%1968 ford mustang%';
```S


Failed to send compressed multipart ingest: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/multipart (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000190A7CCC490>: Failed to resolve \'api.smith.langchain.com\' ([Errno 11001] getaddrinfo failed)"))'))
Content-Length: 3051
API Key: lsv2_********************************************62trace=63bd8df7-7534-4031-832d-be1d5a6c3e9e,id=63bd8df7-7534-4031-832d-be1d5a6c3e9e; trace=63bd8df7-7534-4031-832d-be1d5a6c3e9e,id=374efde8-9dd5-4aa2-9e9e-07536e0b5942; trace=63bd8df7-7534-4031-832d-be1d5a6c3e9e,id=ed8dabc7-e208-49cd-88d2-e0fd3ab673e6; trace=63bd8df7-7534-4031-832d-be1d5a6c3e9e,id=2785646d-b54f-4dfe-879c-bb35f8c613d1; trace=63bd8df7-7534-4031-832d-be1d5a6c3e9e,id=0dca81b6-a6e3-4c3b

now that we able to convert text to sql query then we can now move towards executing it and retrieving the output from the developed query

In [6]:
# since my response format contains text other than sql query I need to filter out the sql query for further execution
import re

def extract_sql_from_response(response: str) -> str:
    """
    Extracts SQL query from a response string containing a code block.
    """
    match = re.search(r"```sql\s*(.*?)\s*```", response, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

# Usage after generating the query
sql_query = extract_sql_from_response(res)
print(sql_query)


SELECT `MSRP` FROM products WHERE `productName` LIKE '%1968 ford mustang%';


*for some LLMs there was no need to create this below runnable but while working cohere it needed that's why I added another layer of output parser so that it won't create issue later*

In [7]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
config = {
    'run_name': 'query_execution'
}
# extract the sql query and execute
if sql_query:
    result = execute_query.invoke(sql_query, config=config)
    print(result)
else:
    print("No SQL query found in response.")


[(Decimal('194.57'),)]


C:\Users\hp\AppData\Local\Temp\ipykernel_8568\213937425.py:3: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


In [8]:
# need to convert the query extraction  function as a runnable
from langchain_core.runnables import RunnableLambda

extract_sql_runnable = RunnableLambda(extract_sql_from_response)


We have both the chains working properly so let's merge them into one chain


In [9]:
chain = generate_query | extract_sql_runnable | execute_query
chain.invoke({'question': 'how many orders aer there'})


'[(326,)]'

In [10]:
# extracting prompt template from the chain
chain.get_prompts()[0].pretty_print()


You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

We are going to develop a simple workflow where we'll take natural query as a input showcase the ouput in structured format.

**Flow:** *question -> sql query generation -> sql query result -> final answer*

In [11]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
)


In [12]:
# this chain will fail because 'rephrased_answer' needs many other inputs which are not there in output of execute query that's why we are using runnablepassthrough.assign() to ensure all the requirements are fulfilled properly
chain = generate_query | extract_sql_runnable | execute_query | rephrased_answer
chain.invoke({'question': "how many orders are there in total"})


NameError: name 'rephrased_answer' is not defined

In [13]:
rephrased_answer = answer_prompt | llm  | StrOutputParser() # output parser will help to get the sql query and return for further process

chain = (
    RunnablePassthrough.assign(query=generate_query).assign( # from generate query we get: user question, sql query
        result = itemgetter('query') | extract_sql_runnable | execute_query # from here we get sql result
    )
    | rephrased_answer # and all the response from runnable passthrough will be sent here to generate final response
)

chain.invoke({'question': "who is the highest paying customer tell it's name and what is the amount"})


"It looks like the SQL query needs to be adjusted to comply with the database's sql_mode=only_full_group_by setting. To fix the error, we should add a GROUP BY clause to the query.\n\nHere's the modified SQL query:\n```sql\nSELECT customers.customerName, SUM(payments.amount) AS highest_amount\nFROM payments\nJOIN customers USING (customerNumber)\nGROUP BY customers.customerName\nORDER BY highest_amount DESC\nLIMIT 1;\n```\n\nThe highest-paying customer is 'John Doe' who has paid a total of '$64000'."

**Adding few-shot examples for better understanding**

LLM may fail if it tackles tough queries

In [14]:
examples = [
    {
        "input": "List all customers in France with a credit limit over 20,000.",
        "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
    },
    {
        "input": "Get the highest payment amount made by any customer.",
        "query": "SELECT MAX(amount) FROM payments;"
    },
    {
        'input': "Show product details for products in the 'motorcycles' product line. ",
        'query': "SELECT * FROM products WHERE productLine = 'Motorcycles';"
    },
    {
        'input': 'Retrieve the names of employees who report to employee number 1002.',
        'query': "SELECT firstName, lastName FROM employees WHERE reportsTo = 1002"
    }, 
    {
        'input': 'List all products with a stock quantity less than 7000',
        'query': "SELECT productName, quantityInStock FROM products WHERE quantityInStock < 700;"
    }
]


In [15]:
# generating few shot prompt template
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    input_variables=['input']
)

print(few_shot_prompt.format(input='how many products are there ?'))


Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;
Human: Show product details for products in the 'motorcycles' product line. 
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: Retrieve the names of employees who report to employee number 1002.
SQLQuery:
AI: SELECT firstName, lastName FROM employees WHERE reportsTo = 1002
Human: List all products with a stock quantity less than 7000
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 700;


right now it's selecting all the examples we want it to select only relevant example from examples dictoinary to help llm generate sql query.

**Dynamic Few Shot Selection**

this will help to select relevant k example which are related to asked user query

In [16]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_cohere import CohereEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    CohereEmbeddings(model='embed-english-v3.0'),
    vectorstore,
    k=2,
    input_keys=['input'],
)

example_selector.select_examples({'input': 'who is the highest paying customer'})


C:\Users\hp\AppData\Local\Temp\ipykernel_8568\2218894559.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma()


[{'input': 'Get the highest payment amount made by any customer.',
  'query': 'SELECT MAX(amount) FROM payments;'},
 {'query': "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;",
  'input': 'List all customers in France with a credit limit over 20,000.'}]

In [17]:
# modifying few shot prompting by selecting top-k related examples as input
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=['input', 'top_k'],
)

print(few_shot_prompt.format(input="what is the count of total products"))


Human: List all products with a stock quantity less than 7000
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 700;
Human: Show product details for products in the 'motorcycles' product line. 
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';


In [18]:
# building new prompt template containing few examples for references
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?",table_info="some table info"))


System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries.
Human: Show product details for products in the 'motorcycles' product line. 
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: List all products with a stock quantity less than 7000
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 700;
Human: How many products are there?


In [19]:
# testing if it works or not
generate_query = create_sql_query_chain(llm, db,final_prompt) # upaded the sql query chain with new prompt template
# rest is same as before
chain = (
    RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter("query") | execute_query
    ) 
    | rephrased_answer
)
chain.invoke({"question": "How many csutomers with credit limit more than 50000"})


'There are 85 customers with a credit limit of more than 50000.'

There is one more issue which persist is now we are providing details of all the tables which we have got in out databases, but fow now we're dealing with very few schema for large it will be quite complicated for LLM to understand importance of each schema. 

We are going to dynamically select relevant schemas which will be needed by LLM to generate SQL query to make our prompt concise and accurate for usage.

For dynamic selection I'm going to create file containing small description about each schema based on that description LLM will be able to select relevant schema information only.

In [20]:
# extracting table descriptions from csv file
import pandas as pd

def get_schema_details():
    table_description = pd.read_csv('mysql_schema_description.csv')
    table_docs = []
    
    table_details = ""
    for index, row in table_description.iterrows():
        table_details = table_details + "Table Name: " + row['Table_Name'] + '\n' + 'Table Description: ' + row['Description'] + "\n\n"
    
    return table_details

table_details = get_schema_details()
print(table_details)


Table Name: productlines
Table Description: Stores information about the different product lines offered by the company, including a unique name, textual description, HTML description, and image. Categorizes products into different lines

Table Name: products
Table Description: Contains details of each product sold by the company, including code, name, product line, scale, vendor, description, stock quantity, buy price, and MSRP Linked to the productlines table.

Table Name: offices
Table Description: Holds data on the company's sales offices, including office code, city, phone number, address, state, country, postal code, and territory. Each office is uniquely identified by its office code.

Table Name: employees
Table Description: Stores information about employees, including number, last name, first name, job title, contact info, and office code. Links to offices and maps organizational structure through the reports To attribute

Table Name: customers
Table Description: Captures dat

In [35]:
from pydantic import BaseModel, Field
from typing import List

class Table(BaseModel):
    """Table in SQL DATABASE"""
    table_name: List[str] = Field(description='Name of the table in SQL Database')


In [36]:
# prompt decribing table details
table_details_prompt = f"""
    Return the names of ALL the SQL tables that MIGHT be relevant to the user Question. \n
    The Tables are:
    {table_details}
    
    REMEMBER: Include ALL POTENTIALLY RELEVANT tables, even if you are not sure that they're needed.
"""

table_chain = llm.with_structured_output(Table)
user_query = "give me details of customer and their order count"
prompt_with_table_details_and_query = f"{table_details_prompt}\n\nUser query: {user_query}"

tables = table_chain.invoke(prompt_with_table_details_and_query)
tables


Table(table_name=['customers', 'orders', 'orderdetails'])

In [67]:
def get_tables(tables) -> List[str]:
    tables = [table for table in tables.table_name]
    return tables


In [68]:
get_tables(tables)


['customers', 'orders', 'orderdetails']

In [77]:
# this chain will be used for table name selection
select_table = (
    {"question": itemgetter("question")}
    | RunnableLambda(lambda x: f"{table_details_prompt}\n\nUser query: {x['question']}")
    | table_chain
    | RunnableLambda(get_tables)
)

tables = select_table.invoke({"question": "give me details of customer and their order count"})
print(tables) 


['customers', 'orders', 'orderdetails']


In [78]:
final_chain_with_dynamic_tables = (
    RunnablePassthrough.assign(table_names_to_use=select_table) 
    | RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter('query') | execute_query
    )
    | rephrased_answer
)


In [79]:
final_chain_with_dynamic_tables.invoke({"question": 'total numbers or customers we have'})


'We have 122 customers.'

Now that it's working properly, there is a problem that it can't answer followup questions. We need to add memory to it so that it answers followup questions.

In [80]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions"),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="messages"),
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?",table_info="some table info",messages=[]))


System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions
Human: Show product details for products in the 'motorcycles' product line. 
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: List all products with a stock quantity less than 7000
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 700;
Human: How many products are there?


**Adding memory feature to it**

In [108]:
from langchain_core.output_parsers import StrOutputParser
import re

# 1. SQL-only parser
class SQLParser(StrOutputParser):
    def parse(self, text: str) -> str:
        # Extract first SQL block or clean out prefix text
        match = re.search(r"(SELECT .*?;)", text, re.DOTALL | re.IGNORECASE)
        if match:
            return match.group(1).strip()
        return text.strip()  # fallback

sql_only_parser = SQLParser()

# 2. Wrap your generate_query chain
generate_query_clean = generate_query | sql_only_parser



In [109]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

generate_query = create_sql_query_chain(llm, db, final_prompt)

chain = (
    RunnablePassthrough.assign(table_names_to_use=select_table) 
    | RunnablePassthrough.assign(query=generate_query_clean).assign(
        result=itemgetter('query') | execute_query
    )
    | rephrased_answer
)


In [110]:
question = "how many customers with order count more than 5"

response = chain.invoke({'question': question, 'messages':history.messages}) # now we're providing previous conversation history to answer follow up questions
response


'There are 2 customers who have placed more than 5 orders.'

In [111]:
history.add_user_message(question)
history.add_ai_message(response)


In [112]:
history


InMemoryChatMessageHistory(messages=[HumanMessage(content='how many customers with order count more than 5', additional_kwargs={}, response_metadata={}), AIMessage(content='There are 2 customers who have placed more than 5 orders.', additional_kwargs={}, response_metadata={})])

In [113]:
follow_up = chain.invoke({'question': 'can you list their names ?', 'messages': history.messages})
follow_up


"Sure! Here's the answer:\n\nThe names are 'Mini Gifts Distributors Ltd.' and 'Euro+ Shopping Channel'."

One Issue persist here is that while asking follow up question, it's giving all the possible schemas as relevant schemas because that prompt doesn't have access to history so we need to integrate the history there also so that while asking follow up questions it tells only relevant schema names.